In [1]:
import os
import sys
sys.path.insert(1,'/home/bwhiteak/ChaosESN/ESN_utils/')

import numpy as np
import matplotlib.pyplot as plt
import rc_tools as rct
import rc_matrices as rcm
import rc_analysis as rca
from scipy.optimize import fsolve
import time

import pdb

In [3]:
def rank_along_trajectory(wr, wi, a, g, forcing, n, k, tolerance):
    #pdb.set_trace()
    T = forcing.shape[0]
    ranks = np.zeros(T)
    conds = np.zeros(T)
    radii = np.zeros(T)
    def Func(x):
        return np.squeeze(-x.reshape(n,1) + (1-a)*x.reshape(n,1) +\
                          a*g*np.tanh(np.dot(wr,x.reshape(n,1)) +\
                         (np.dot(wi,ueq.reshape(k,1)).reshape(n,1))))
    for i in range(T):
        start = time.time()
        ueq = forcing[i]
        x0 = np.ones((n,1))*.1
        xeq = (fsolve(Func,x0)).reshape(n,1)
        A = rca.leaky_jacobian(xeq, ueq.reshape(k,1), a, g, Wi, Wr)
        radii[i] = np.max(np.abs(rca.eig_spectrum(A)))
        conds[i] = rca.condition_number(A)[0]
        B = rca.partial_u(xeq, ueq.reshape(k,1), a, g, Wi, Wr)
        Cn = rca.reachable_matrix(A, B)
        Cn = Cn/np.max(rca.eig_spectrum(Cn))
        ranks[i] = rca.rank(Cn, tolerance)
        if(i%10==0):
            runtime = time.time()-start
            print(f'Time: {runtime:.2f} Column: {i} Rank: {ranks[i]} CondNum: {conds[i]:.2f}')
    return ranks, conds, radii

def rank_curve(cplus, tols):
    rank_tols = []
    for i in tols:
        rank_tols.append(rca.rank(cplus, i))
    return np.array(rank_tols)

In [25]:
sizes = [50,100,200,300,400,500,600,700,800,900,1000]
target_path = ['../Targets/Xsignal.npy',
              '../Targets/Zsignal.npy',
              '../Targets/MGsignal.npy',
              '../Targets/L3Dsignal.npy']
model_dir = ['../X_Saved_Models',
             '../Z_Saved_Models',
             '../MG_Saved_Models',
             '../L3D_Saved_Models']
SIGNUM = 3
K=1
tols = [1/10**x for x in range(0,30)]

rank_arr = np.zeros(len(sizes))
counter = 0
for n in sizes:
    forcing = np.load(target_path[SIGNUM], allow_pickle=True)
    wnet_dir = 'best/Wnet'
    wnet_str  = f'L3D_{n}_Wnet.npy'
    fwpath = os.path.join(model_dir[SIGNUM],wnet_dir,wnet_str)

    param_dir = 'best/Params'
    param_str = f'L3D_{n}_Params.npy'
    fppath = os.path.join(model_dir[SIGNUM],param_dir,param_str)
    p = np.load(fppath, allow_pickle=True)
    Wr, Wi = rca.get_mats(fwpath, K,n)
    a,g = p[0],p[2]
    x0 = np.zeros((n,1))
    u0 = np.zeros((1,1))
    A = rca.leaky_jacobian(x0, u0, a, g, Wi, Wr)
    B = rca.partial_u(x0, u0, a, g, Wi, Wr)
    Cn = np.nan_to_num(rca.reachable_matrix(A,B))
    Cn = Cn/np.max(rca.eig_spectrum(Cn))
    
    rank_by_tol = []
    for i in tols:
        rank_by_tol.append(rca.rank(Cn,i))
    argmx = np.argmax(np.gradient(rank_by_tol))
    rank_arr[counter] = rank_by_tol[argmx-1]
    counter += 1
print(rank_arr)
np.save('./NBoutputs/mmL3Dranks', rank_arr)

[  4.  85.  31. 101. 154. 221. 147. 353. 110.  88. 125.]
